In [1]:
import os
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import h5py
import cv2
%matplotlib inline

In [2]:
labels = []

In [3]:
filename = None

for filename in range(1, 3065):
    with h5py.File('./brain_tumor_dataset/imageData/{}.mat'.format(filename), 'r') as f:
        label = f['cjdata']['label'][0][0] - 1
        labels.append(int(label))

In [4]:
for i in range(len(labels)):
    if(labels[i]==0):
        labels[i] = "glioma"
    elif(labels[i]==1):
        labels[i] = "meningioma"
    else:
        labels[i] = "pituitary"

In [5]:
def importing_data(path):
    sample = []
    for filename in os.listdir(path):
        if "jpg" in filename:
            sample.append(filename)
    return sample

In [6]:
path = './brain_tumor_dataset/bt_images/'
train = importing_data(path)

train_data = pd.DataFrame({'image':train, 'label': labels})

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val = train_test_split(train_data, test_size = 0.1, shuffle = True, random_state = 42)

In [8]:
IMG_SIZE = 64
size = (IMG_SIZE,IMG_SIZE)

In [9]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Conv2D , MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [20]:
inputs = Input(shape = (64, 64, 1))
conv1 =  Conv2D(32, (3,3), activation = 'selu', padding = 'same', strides=(1,1), input_shape = [64,64,1])(inputs)
pool1 = MaxPooling2D(2)(conv1)
conv2 = Conv2D(64, (3,3), activation = 'selu', padding = 'same', strides=(1,1))(pool1)
pool2 = MaxPooling2D(2)(conv2)
conv3 = Conv2D(64, (3,3), activation = 'selu', padding = 'same', strides=(1,1))(pool2)
pool3 = MaxPooling2D(2)(conv3)
conv4 = Conv2D(128,(3,3), activation = 'selu', padding = 'same', strides=(1,1))(pool3)
pool4 = MaxPooling2D(2)(conv4)
conv5 = Conv2D(256,(3,3), activation = 'selu', padding = 'same', strides=(1,1))(pool4)
pool5 = MaxPooling2D(2)(conv5)
flatten = Flatten()(pool5)
dense1 = Dense(32, activation = 'selu')(flatten)
dropout1 = Dropout(0.25)(dense1)
outputs = Dense(3, activation = 'softmax')(dropout1)
model = Model(inputs=inputs, outputs=outputs)


model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0     

In [20]:
datagen = ImageDataGenerator(rescale = 1./255)

train_set = datagen.flow_from_dataframe(X_train,
                                        directory = './brain_tumor_dataset/bt_images/',
                                        x_col = 'image',
                                        y_col = 'label',
                                        target_size = size,
                                        color_mode = 'grayscale',
                                        class_mode = 'categorical',
                                        batch_size = 1,
                                        shuffle = True,
                                        interpolation = 'bilinear')

val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

    

model.compile(optimizer="adamax",loss='categorical_crossentropy',metrics=['accuracy']) 
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)
checkpoint_cb = ModelCheckpoint("./models/bayesian_selu_adamax.h5", save_best_only=True)
history = model.fit(train_set, validation_data = val_set, epochs= 30, callbacks=[checkpoint_cb, es])   

Found 2757 validated image filenames belonging to 3 classes.
Found 307 validated image filenames belonging to 3 classes.
Epoch 1/30
2757/2757 [==============================] - 23s 8ms/step - loss: 0.6745 - accuracy: 0.7145 - val_loss: 0.4801 - val_accuracy: 0.8143
Epoch 2/30
2757/2757 [==============================] - 30s 11ms/step - loss: 0.4317 - accuracy: 0.8248 - val_loss: 0.4007 - val_accuracy: 0.8274
Epoch 3/30
2757/2757 [==============================] - 43s 16ms/step - loss: 0.2269 - accuracy: 0.9202 - val_loss: 0.2423 - val_accuracy: 0.9186
Epoch 4/30
2757/2757 [==============================] - 42s 15ms/step - loss: 0.1123 - accuracy: 0.9623 - val_loss: 0.2103 - val_accuracy: 0.9283
Epoch 5/30
2757/2757 [==============================] - 48s 18ms/step - loss: 0.0506 - accuracy: 0.9840 - val_loss: 0.2714 - val_accuracy: 0.9218
Epoch 6/30
2757/2757 [==============================] - 44s 16ms/step - loss: 0.0320 - accuracy: 0.9902 - val_loss: 0.2576 - val_accuracy: 0.9121
Rest

In [10]:
def preprocess(img):
    img = img.astype(np.float32)
    img /= 255.0
    img = np.expand_dims(img,0)
    return img

In [11]:
val = X_val["image"].values.tolist()

In [12]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian_selu_adamax.h5"

In [13]:
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "adamax", loss='categorical_crossentropy', metrics =['accuracy'])

In [14]:
def perturb(x):
    adv_img = orig.copy()

    pixs = np.array(np.split(x, len(x)/3)).astype(int)
    loc = (pixs[:, 0], pixs[:,1])
    val = pixs[:, 2]
    adv_img[loc] = val
    adv_img = np.expand_dims(adv_img,0)
    return adv_img

In [15]:
def optimize(x):
    adv_img = perturb(x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    pred_orig = np.argmax(prob_orig)

    return prob[0][pred_orig]

In [16]:
pred_adv = 0
prob_adv = 0
ct = 0
def callback(x, convergence):
    global pred_adv, prob_adv, ct
    adv_img = perturb(x)


    prob = keras_model.predict(preprocess(adv_img[0]))

    pred_adv = np.argmax(prob)
    prob_adv = prob[0][pred_adv]
    if pred_adv != pred_orig:
        ct = ct+1
        return True

In [28]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)


    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

    if(pred_orig != pred_adv):
        cv2.imwrite('./bayesian_outputs_selu_adamax/'+filename, adv_img[0])

Prob [pituitary]: 0.991116 --> Prob[pituitary]: 0.961557
Prob [glioma]: 0.950651 --> Prob[glioma]: 0.740844
Prob [pituitary]: 0.999937 --> Prob[pituitary]: 0.998750
Prob [pituitary]: 0.993387 --> Prob[pituitary]: 0.971507
Prob [pituitary]: 0.983965 --> Prob[meningoma]: 0.516241
Prob [glioma]: 0.973712 --> Prob[glioma]: 0.929503
Prob [meningoma]: 0.998619 --> Prob[meningoma]: 0.995244
Prob [pituitary]: 0.929718 --> Prob[meningoma]: 0.545038
Prob [glioma]: 0.986935 --> Prob[glioma]: 0.949418
Prob [pituitary]: 0.996337 --> Prob[pituitary]: 0.970607
Prob [meningoma]: 0.974561 --> Prob[meningoma]: 0.846300
Prob [meningoma]: 0.999974 --> Prob[meningoma]: 0.999964
Prob [glioma]: 0.802842 --> Prob[glioma]: 0.530252
Prob [pituitary]: 0.998433 --> Prob[pituitary]: 0.987577
Prob [pituitary]: 0.839608 --> Prob[meningoma]: 0.538543
Prob [meningoma]: 0.999753 --> Prob[meningoma]: 0.999391
Prob [meningoma]: 0.922049 --> Prob[meningoma]: 0.770620
Prob [meningoma]: 0.993109 --> Prob[meningoma]: 0.98073

Prob [meningoma]: 0.974791 --> Prob[meningoma]: 0.884188
Prob [pituitary]: 0.933910 --> Prob[pituitary]: 0.642470
Prob [meningoma]: 0.999741 --> Prob[meningoma]: 0.999160
Prob [meningoma]: 0.992106 --> Prob[meningoma]: 0.969946
Prob [pituitary]: 0.860006 --> Prob[meningoma]: 0.694526
Prob [pituitary]: 0.999525 --> Prob[pituitary]: 0.997421
Prob [pituitary]: 0.999076 --> Prob[pituitary]: 0.995277
Prob [meningoma]: 0.905032 --> Prob[meningoma]: 0.607582
Prob [glioma]: 0.928534 --> Prob[glioma]: 0.733375
Prob [meningoma]: 0.983722 --> Prob[meningoma]: 0.946020
Prob [glioma]: 0.896521 --> Prob[glioma]: 0.555605
Prob [meningoma]: 0.776912 --> Prob[meningoma]: 0.662016
Prob [meningoma]: 0.997044 --> Prob[meningoma]: 0.985396
Prob [glioma]: 0.766962 --> Prob[meningoma]: 0.533148
Prob [pituitary]: 0.999439 --> Prob[pituitary]: 0.992079
Prob [glioma]: 0.800189 --> Prob[glioma]: 0.519871
Prob [meningoma]: 0.975918 --> Prob[meningoma]: 0.901952
Prob [meningoma]: 0.555610 --> Prob[pituitary]: 0.67

Prob [meningoma]: 0.999972 --> Prob[meningoma]: 0.999949
Prob [glioma]: 0.997714 --> Prob[glioma]: 0.992323
Prob [meningoma]: 0.940289 --> Prob[meningoma]: 0.731853
Prob [glioma]: 0.999785 --> Prob[glioma]: 0.999580
Prob [pituitary]: 0.996270 --> Prob[pituitary]: 0.986937
Prob [pituitary]: 0.977420 --> Prob[pituitary]: 0.585702
Prob [glioma]: 0.997458 --> Prob[glioma]: 0.993233
Prob [glioma]: 0.789429 --> Prob[meningoma]: 0.539173
Prob [pituitary]: 0.992860 --> Prob[pituitary]: 0.948443
Prob [meningoma]: 0.651985 --> Prob[pituitary]: 0.425610
Prob [pituitary]: 0.986791 --> Prob[pituitary]: 0.950220


In [31]:
print(ct)

43


In [17]:
datagen = ImageDataGenerator(rescale = 1./255)
adv_set = datagen.flow_from_dataframe(X_val,
                                      directory = './bayesian_outputs_selu_adamax/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 43 validated image filenames belonging to 3 classes.


D:\Softwares\anaconda\envs\tf-gpu\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:279: UserWarning: Found 264 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


In [14]:
predict = keras_model.evaluate(adv_set)

43/43 [==============================] - 45s 79ms/step - loss: 0.7840 - accuracy: 0.4186


In [15]:
path = './bayesian_outputs_selu_adamax/'
adv = importing_data(path)
adv_df = pd.DataFrame({'image':adv})
val_df = X_val[~X_val.image.isin(adv_df.image)]

In [18]:
val_set = datagen.flow_from_dataframe(X_val,
                                      directory = './brain_tumor_dataset/bt_images/',
                                      x_col = 'image',
                                      y_col = 'label',
                                      target_size = size,
                                      color_mode = 'grayscale',
                                      class_mode = 'categorical',
                                      batch_size = 1,
                                      shuffle = True,
                                      interpolation = 'bilinear')

Found 307 validated image filenames belonging to 3 classes.


In [17]:
keras_model.evaluate(val_set)

264/264 [==============================] - 11s 40ms/step - loss: 0.1600 - accuracy: 0.9583


[0.1599799394607544, 0.9583333134651184]

In [21]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=2, restore_best_weights=True, verbose=1)

checkpoint_cb = ModelCheckpoint("./models/bayesian_selu_adamax_adversarial.h5", save_best_only=True)

history_adv = keras_model.fit(adv_set, validation_data = val_set, epochs= 50, batch_size = 1, callbacks=[es, checkpoint_cb])   

Epoch 1/50
43/43 [==============================] - 61s 381ms/step - loss: 1.4234 - accuracy: 0.6047 - val_loss: 0.3217 - val_accuracy: 0.8795
Epoch 2/50
43/43 [==============================] - 5s 116ms/step - loss: 0.1585 - accuracy: 0.9535 - val_loss: 0.2985 - val_accuracy: 0.8567
Epoch 3/50
43/43 [==============================] - 5s 123ms/step - loss: 0.0371 - accuracy: 0.9767 - val_loss: 0.2329 - val_accuracy: 0.8958
Epoch 4/50
43/43 [==============================] - 5s 116ms/step - loss: 0.0183 - accuracy: 1.0000 - val_loss: 0.1977 - val_accuracy: 0.9153
Epoch 5/50
43/43 [==============================] - 5s 124ms/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.2379 - val_accuracy: 0.8990
Epoch 6/50
43/43 [==============================] - 5s 120ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.2315 - val_accuracy: 0.9055
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping


In [22]:
class_names = {0: "glioma", 1: "meningoma", 2:"pituitary"}
d = 1
iters = 600
popsize = 10
model_path = "./models/bayesian_selu_adamax_adversarial.h5"
import keras 
keras_model = keras.models.load_model(model_path)
keras_model.compile(optimizer = "adamax", loss='categorical_crossentropy', metrics =['accuracy'])

In [23]:
from scipy.optimize import differential_evolution

for filename in val:
    pred_adv = 0
    prob_adv = 0
    orig = cv2.imread(os.path.join("./brain_tumor_dataset/bt_images/",filename))
    orig = cv2.cvtColor(orig, cv2.COLOR_BGR2GRAY)
    orig = cv2.resize(orig, size)
    
    img = orig.copy()
    img = img.reshape(64,64,1)
    img = np.expand_dims(img,0)
    shape = img.shape
    
    
    prob_orig = keras_model.predict(preprocess(img[0]))
    pred_orig = np.argmax(prob_orig)

    bounds = [(0, shape[1]-1), (0, shape[2]-1), (0, 255)] * d
    result = differential_evolution(optimize, bounds, maxiter=iters, popsize=popsize, tol=1e-5, callback=callback)

    adv_img = perturb(result.x)

    prob = keras_model.predict(preprocess(adv_img[0]))
    print('Prob [%s]: %f --> Prob[%s]: %f ' %(class_names[pred_orig], prob_orig[0][pred_orig], class_names[pred_adv], prob_adv))

Prob [meningoma]: 0.446052 --> Prob[pituitary]: 0.410345 
Prob [glioma]: 0.965465 --> Prob[glioma]: 0.912671 
Prob [pituitary]: 0.996268 --> Prob[pituitary]: 0.967461 
Prob [meningoma]: 0.813233 --> Prob[meningoma]: 0.642446 
Prob [pituitary]: 0.998457 --> Prob[pituitary]: 0.978891 
Prob [glioma]: 0.998824 --> Prob[glioma]: 0.997354 
Prob [meningoma]: 0.968165 --> Prob[meningoma]: 0.937956 
Prob [pituitary]: 0.999998 --> Prob[pituitary]: 0.999994 
Prob [glioma]: 0.988434 --> Prob[glioma]: 0.975164 
Prob [meningoma]: 0.619598 --> Prob[pituitary]: 0.475870 
Prob [meningoma]: 0.999019 --> Prob[meningoma]: 0.998307 
Prob [meningoma]: 0.999979 --> Prob[meningoma]: 0.999971 
Prob [glioma]: 0.712843 --> Prob[meningoma]: 0.499842 
Prob [pituitary]: 0.760963 --> Prob[meningoma]: 0.531651 
Prob [meningoma]: 0.998090 --> Prob[meningoma]: 0.995755 
Prob [meningoma]: 0.999861 --> Prob[meningoma]: 0.999752 
Prob [meningoma]: 0.993294 --> Prob[meningoma]: 0.983054 
Prob [meningoma]: 0.999228 --> Prob

Prob [meningoma]: 0.999903 --> Prob[meningoma]: 0.999730 
Prob [meningoma]: 0.997673 --> Prob[meningoma]: 0.994577 
Prob [meningoma]: 0.837841 --> Prob[meningoma]: 0.726479 
Prob [meningoma]: 0.872735 --> Prob[meningoma]: 0.684318 
Prob [meningoma]: 0.735895 --> Prob[meningoma]: 0.563084 
Prob [meningoma]: 0.999759 --> Prob[meningoma]: 0.999555 
Prob [meningoma]: 0.993351 --> Prob[meningoma]: 0.972288 
Prob [pituitary]: 0.992013 --> Prob[pituitary]: 0.928539 
Prob [pituitary]: 0.784546 --> Prob[meningoma]: 0.516927 
Prob [pituitary]: 0.873721 --> Prob[pituitary]: 0.725222 
Prob [meningoma]: 0.742326 --> Prob[meningoma]: 0.518443 
Prob [glioma]: 0.900283 --> Prob[glioma]: 0.796672 
Prob [meningoma]: 0.958389 --> Prob[meningoma]: 0.885485 
Prob [glioma]: 0.994768 --> Prob[glioma]: 0.978406 
Prob [meningoma]: 0.911386 --> Prob[meningoma]: 0.845252 
Prob [meningoma]: 0.540159 --> Prob[glioma]: 0.609514 
Prob [glioma]: 0.999808 --> Prob[glioma]: 0.999551 
Prob [pituitary]: 0.969976 --> Prob

Prob [meningoma]: 0.996167 --> Prob[meningoma]: 0.992344 
Prob [meningoma]: 0.987803 --> Prob[meningoma]: 0.970550 
Prob [meningoma]: 0.984780 --> Prob[meningoma]: 0.968808 
Prob [pituitary]: 0.801951 --> Prob[meningoma]: 0.562479 
Prob [meningoma]: 0.821034 --> Prob[meningoma]: 0.624887 
Prob [meningoma]: 0.999658 --> Prob[meningoma]: 0.999343 
Prob [glioma]: 0.998990 --> Prob[glioma]: 0.997703 
Prob [meningoma]: 0.995915 --> Prob[meningoma]: 0.989887 
Prob [glioma]: 0.999831 --> Prob[glioma]: 0.999709 
Prob [pituitary]: 0.868470 --> Prob[pituitary]: 0.627297 
Prob [meningoma]: 0.701708 --> Prob[pituitary]: 0.501539 
Prob [glioma]: 0.999937 --> Prob[glioma]: 0.999917 
Prob [glioma]: 0.984756 --> Prob[glioma]: 0.934992 
Prob [pituitary]: 0.925940 --> Prob[pituitary]: 0.729431 
Prob [meningoma]: 0.998517 --> Prob[meningoma]: 0.996899 
Prob [meningoma]: 0.858299 --> Prob[meningoma]: 0.646968 


In [24]:
print(ct)

40
